### Hashing 

A hash function is a function that takes an inpute and deterministically converts it to an integer that is less than a fized size set by the programmer. Inputs are called **keys** and the same input will always be converted to the same integer.

**Check if the sentence is Pangram**
- A pangram is a sentence where every letter of the English alphabet appears at least once.

In [1]:
#Q: Given a string sentence containing only lowercase English letters, return true if sentence is a pangram or false otherwise
def is_pangram(sentence):
    seen = set()

    for i in sentence:
        seen.add(i)
        if len(seen) == 26:
            return True 
        
    return len(seen) == 26 

if __name__ == '__main__':
    sentence = "thequickbrownfoxjumpsoverthelazydog"
    print(is_pangram(sentence))

True


## Separate Chaining 

### Create A HashTable Or Hash Function/ Hash Map: Separate Chaining

Python implementation of a simple hash table (also known as a hash map) using **separate chaning** to handle **collision** including basic operators or methods such as **set**, **get** and **delete** 

In [2]:
class HashTable:
    def __init__(self, size=100):
        """ 
        Initialize the table with a fixed size.
        Each slot in the table contains a list to handle collision via chaining.
        """
        self.size = size 
        self.table = [[] for _ in range(self.size)]

    def _hash(self, key):
        """
        Generate a hash for the given key.
        Using the python built-in hash function 
        """
        return hash(key) % self.size 
    
    def set(self, key, value):
        """ 
        Insert a key-value pair into the hash table 
        If the key already exist, update the value
        """
        index =  self._hash(key)
        #check if the key already exist and update the value
        for pair in self.table[index]:
            if pair[0] == key:
                pair[1] = value
                print(f"Updated key '{key}' with value '{value}'.")
                return 
        
        #If the key doesn't exist, append new key-value pair
        self.table[index].append([key, value])
        print(f"Inserted key '{key}' with value '{value}'.")

    def get(self, key):
        """
        Retrieve the value associated with the given key 
        Returns None if the key is not Found 
        """
        index = self._hash(key)
        for pair in self.table[index]:
            if pair[0] == key:
                print(f"Found key '{key}' with value '{pair[1]}'.")
                return pair[1]
        
        print(f"Key '{key}' not Found")
        return None
    
    def delete(self, key):
        """
        Remove the key-value pair associated with the given key from the hash table  
        """
        index = self._hash(key)
        for i, pair in enumerate(self.table[index]):
            if pair[0] == key:
                del self.table[index][i]
                print(f"Deleted key '{key}'.")
                return 
        print(f"Key '{key }' not found. Nothing to delete.")

    def __str__(self):
        """ 
        For debugging: Return a string representation of the hash table.
        """
        table_str = ""
        for i, bucket in enumerate(self.table):
            if bucket:
                table_str += f"Bucket {i}: " + ", ".join([f"{k}: {v}" for k, v in bucket]) + "\n"
        return table_str if table_str else "hash Table is empty."
    

if __name__ == '__main__':
    ht = HashTable(size=10)

    #Insert key-value pairs
    ht.set("apple", 1)
    ht.set("banana", 2)
    ht.set("orange", 3)
    ht.set("apple", 10)  # Update existing key

    # Retrieve values
    print(ht.get("apple"))   # Output: 10
    print(ht.get("banana"))  # Output: 2
    print(ht.get("grape"))   # Output: None

    # Delete a key
    ht.delete("banana")
    print(ht.get("banana"))  # Output: None

    # Print the current state of the hash table
    print(ht)



Inserted key 'apple' with value '1'.
Inserted key 'banana' with value '2'.
Inserted key 'orange' with value '3'.
Updated key 'apple' with value '10'.
Found key 'apple' with value '10'.
10
Found key 'banana' with value '2'.
2
Key 'grape' not Found
None
Deleted key 'banana'.
Key 'banana' not Found
None
Bucket 0: apple: 10
Bucket 5: orange: 3



**Explanation**:

- **Collision Handling**: This implementation uses **separate chaining**, where each bucket contains a list of key-value pairs. When multiple keys hash to the same bucket, they are stored in the list for that bucket.

- **Dynamic Resizing**: The above HashTable does not handle dynamic resizing of the hash table. For a production-ready hash table, we   will need to implement resizing (for example doubling the size) when the load factor exceeds a certain threshold to maintain efficient operations.

**Performance**
- **Average Case**: $O(1)$ for `set`, `get` and `delete` operations.
- **Worse Case**: $O(n)$ when all key collide into the same bucket.

### Hash Table Implementation with Separate Chaining to Include Dynamic Resizing 

In [3]:
class HashTable:
    def __init__(self, size=100):
        """ 
        Initialize the hash table with a fixed size 
        Each slot in the table contains a list to handle collision via chaining 
        """
        self.size = size 
        self.table = [[] for _ in range(self.size)] 
        self.count = 0 # Number of elements in the table 
        self.load_factor_threshold = 0.75 # Resize when load factor exceeds 0.75

    def _hash(self, key):
        """ 
        Use Python's built-in hash function to generate a hash for the given key 
        Ensures the hash is within the table bounds
        """
        return hash(key) % self.size 
    
    def _resize(self):
        """ 
        Resize the table when the load factor exceeds the threshold 
        This involves creating a new table with double in size and rehashing all the keys
        """
        new_size = self.size * 2 
        new_table = [[] for _ in range(new_size)]

        #Rehash all items into the new table 
        for bucket in self.table:
            for key, value in bucket:
                new_index = hash(key) % new_size # Rehash with new table size 
                new_table[new_index].append([key, value])

        #Replace the old table with the new table 
        self.table = new_table
        self.size = new_size
        print(f"Resized hash table to new size {new_size}")
    
    def set(self, key, value):
        """ 
        First check if the key already exists and update the value 
        If not exists insert key-value pairs 
        """
        index = self._hash(key)
        for pair in self.table[index]:
            if pair[0] == key:
                pair[1] = value 
                print(f"Updated key '{key}' with value '{value}' .")
                return 
        
        #Insert key-value pair 
        self.table[index].append([key, value])
        self.count += 1
        print(f"Inserted key '{key}' with value '{value}'.")

        # Check the load factor threshold and resize if neccessary 
        if self.count / self.size > self.load_factor_threshold:
            self._resize()

    def get(self, key):
        """ 
        Retrieve the value assocaited with the given key
        Returns None if the key is not found 
        """
        index = self._hash(key) 

        for pair in self.table[index]:
            if pair[0] == key:
                print(f"Found key '{key}' with value '{pair[1]}'.")
                return pair[1]
        
        print(f"key '{key}' not found.")
        return None 
    
    def delete(self, key):
        """ 
        Remove the key-value pair associated with the given key from the hash table.
        """
        index = self._hash(key)
        for i, pair in enumerate(self.table[index]):
            if pair[0] == key:
                del self.table[index][i]
                self.count -= 1
                print(f"Deleted key '{key}'.")
                return 
        print(f"key '{key}' not found. Nothing to delete.")

    def __str__(self):
        """ 
        For Debugging: Return string representation of items in hash table 
        """
        table_str = ""
        for i, bucket in enumerate(self.table):
            if bucket:
                table_str += f"Bucket {i}: " + ", ".join([f"{k}: {v}" for k, v in bucket]) + "\n"
        return table_str if table_str else "Hash Table is empty."
    
if __name__ == '__main__':
    ht = HashTable(size=10) #initialize the hash table with 10 buckets 

    #Insert key-value pairs 
    ht.set("apple", 1)
    ht.set("banana", 2)
    ht.set("orange", 3)
    ht.set("apple", 10)  # Update existing key

    # Trigger resizing by adding more elements
    ht.set("grape", 4)
    ht.set("melon", 5)
    ht.set("berry", 6)
    ht.set("peach", 7)

    # Retrieve values
    print(ht.get("apple"))   # Output: 10
    print(ht.get("banana"))  # Output: 2
    print(ht.get("grape"))   # Output: 4

    # Delete a key
    ht.delete("banana")
    print(ht.get("banana"))  # Output: None

    # Print the current state of the hash table
    print(ht)

Inserted key 'apple' with value '1'.
Inserted key 'banana' with value '2'.
Inserted key 'orange' with value '3'.
Updated key 'apple' with value '10' .
Inserted key 'grape' with value '4'.
Inserted key 'melon' with value '5'.
Inserted key 'berry' with value '6'.
Inserted key 'peach' with value '7'.
Found key 'apple' with value '10'.
10
Found key 'banana' with value '2'.
2
Found key 'grape' with value '4'.
4
Deleted key 'banana'.
key 'banana' not found.
None
Bucket 0: apple: 10
Bucket 3: melon: 5, peach: 7
Bucket 4: berry: 6
Bucket 5: orange: 3
Bucket 8: grape: 4



**Explanation**

- **Dynamic Resizing**: Define `_resize()` function to rehash items into a new table of double size using the new table size `hash(key) % new_size`

- **Load Factor Check:** After each insertion `set` the current load factor (number of elements divided by the tabls size) is checked. If it exceeds the threshold (0.75 in this case), the table is resized to maintain efficient access time.

### Hash Table with Manual **hash function** using separate chaining  to handle collision  

In [6]:
class HashTable:
    def __init__(self, size=100):
        """ 
        Initialize the table with a fixed size 
        Each slot in the table contains a list to handle collision via chaining 
        """
        self.size = size 
        self.table = [[] for _ in range(self.size)]

    def _hash(self, key):
        """
        Generate a hash for the given key
        Manaully create a hash function using the sum ASCII values  
        """
        hash_value = 0 
        for char in key:
            hash_value += ord(char) #Sum the ASCII values of the characters in the key
        return hash_value % self.size #Ensures the hash is within the table bounds 
    
    def set(self, key, value):
        """
        Insert a key-value pair into the hash table
        If the key already existed update the value 
        """
        index = self._hash(key)
        for pair in self.table[index]:
            #check if the key already exist, update the value 
            if pair[0] == key:
                pair[1] = value 
                print((f"Updated key '{key}' with value '{value}'."))
                return 

        self.table[index].append([key, value])
        print(f"Inserted key '{key}' with value '{value}'.")

    def get(self, key): 
        """ 
        Retrieve value associated with a given key 
        Returns None if key not found 
        """
        index = self._hash(key)
        for pair in self.table[index]:
            if pair[0] == key:
                print(f"The value for key '{key}' is '{pair[1]}'.")
                return pair[1]
            
        print(f"key '{key}' not found")
        return None # Key not found 
    
    def delete(self, key):
        """ 
        Remove or delete key-value pair associated with a given key from the hash table
        """
        index = self._hash(key)
        
        for i, pair in enumerate(self.table[index]):
            if pair[0] == key:
                del self.table[index][i]
                print(f"Deleted key '{key}'.")
                return 
        
        print(f"Key '{key}' not found. Nothing to delete.")

    def __str__(self):
        """ 
        For Debugging, return a string representation of the hash table buckets
        """
        table_str = "" 
        for i, bucket in enumerate(self.table):
            if bucket:
                table_str += f"Bucket{i}: " + ", ".join([f"{k}: {v}" for k, v in bucket]) + "\n"
        return table_str if table_str else "Hash Table is empty"
    
if __name__ == '__main__':
    hash_bucket = HashTable(size=10)
    hash_bucket.set("apple", 1)
    hash_bucket.set("banana", 2)
    hash_bucket.set("orange", 3)
    hash_bucket.set("apple", 10) #updated apple value with 10

    #Get item by key 
    hash_bucket.get("apple")
    hash_bucket.get("banana")
    hash_bucket.get("orange")

    #Delete item by key
    hash_bucket.delete("banana")
    hash_bucket.get("banana")

Inserted key 'apple' with value '1'.
Inserted key 'banana' with value '2'.
Inserted key 'orange' with value '3'.
Updated key 'apple' with value '10'.
The value for key 'apple' is '10'.
The value for key 'banana' is '2'.
The value for key 'orange' is '3'.
Deleted key 'banana'.
key 'banana' not found


#### Python HashTable Collision Handling with Separate Chaining 
- Use Python Log Levels for Erro Handling

In [16]:
import logging

#Set logging configurations 
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

class HashTable:
    def __init__(self, size=100):
        """ 
        Initialize table with a fized size
        Each slot in the table contains list to handle collision
        """
        self.size = size 
        self.table = [[] for _ in range(self.size)]

    def _hash(self, key):
        """ 
        Create a hash function to map object keys
        """
        hash_value = 0 
        for char in key:
            hash_value += ord(char)
        return hash_value % self.size
    
    def set(self, key, value):
        """ 
        Check if a key-value already exists and update the value 
        Update hash table with new key-value pair
        """
        index = self._hash(key)
        found = False
        for pair in self.table[index]:
            if pair[0] == key:
                pair[1] = value
                found = True 
                log.info(f"Updated key '{key}' with value '{value}'.")
                return 
        if not found:
            self.table[index].append([key, value])
            log.info(f"Inserted key '{key}' with value '{value}' into the hash table.")
    
    def get(self, key):
        """ 
        Retrieve item by keys in the hash table
        """
        index = self._hash(key)
        for pair in self.table[index]:
            if pair[0] == key:
                log.info(f"The value of key '{key}' is '{pair[1]}'.")
                return pair[1]
            
        log.info(f"The value of the item '{key}' not found.")
        return None
    
    def delete(self, key):
        """
        Delete item by key in the hash table 
        """
        index = self._hash(key)
        for i, pair in enumerate(self.table[index]):
            if pair[0] == key:
                del self.table[index][i]
                log.info(f"Deleted item key '{key}' from hash table.")
                return 
            
        log.info(f"Item with key '{key}' not found in hash table.")
        return None
    
    def __str__(self) -> str:
        """ 
        For dugging: return a string representation of the hash table
        """
        table_str = "" 
        for i, bucket in enumerate(self.table):
            if bucket:
                table_str += f"Bucket {i}: " + ", ".join([f" {k}: {v}" for k , v in bucket]) + "\n"
        return table_str if table_str else "Hash Table is empty"
    
if __name__ == '__main__':
    hash_buck = HashTable(size=10)

    #Set items in hash table 
    hash_buck.set("apple", 1)
    hash_buck.set("banana", 2)
    hash_buck.set("orange", 3)
    hash_buck.set("apple", 10)

    #Get item 
    hash_buck.get("apple")
    hash_buck.get("banana")
    hash_buck.get("orange")

    #Delete item by key
    hash_buck.delete("banana")
    hash_buck.get("banana")

INFO:__main__:Inserted key 'apple' with value '1' into the hash table.
INFO:__main__:Inserted key 'banana' with value '2' into the hash table.
INFO:__main__:Inserted key 'orange' with value '3' into the hash table.
INFO:__main__:Updated key 'apple' with value '10'.
INFO:__main__:The value of key 'apple' is '10'.
INFO:__main__:The value of key 'banana' is '2'.
INFO:__main__:The value of key 'orange' is '3'.
INFO:__main__:Deleted item key 'banana' from hash table.
INFO:__main__:The value of the item 'banana' not found.


#### Hash Table With Dynamic Resizing Using Separate Chaining and Manual Hash Function 

- Use Python Log Level for Error Handling 

In [29]:
import logging

#Set logging configuration
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

class HashTable:
    def __init__(self, size=100):
        """ 
        Initialize table with a fixed size
        Each slot in the table contains a list for handling collision 
        """
        self.size = size
        self.table = [[] for _ in range(self.size)]
        self.count =  0 # Count number of elements in the hash table 
        self.load_factor_threshold = 0.75 #If the hash table exceeds the load factor threshold, resize the hash table 

    def _hash(self, key):
        """ 
        Create a hash function using ASCII sum value
        Ensures the hash is within the table bounds 
        """
        hash_value = 0 
        for char in key:
            hash_value += ord(char)
        return hash_value % self.size
    
    def _resize(self):
        """ 
        Resize the hash table if it exceeds the load factor threshold 
        This involves creating a new table double in size and rehashing all elements in the table 
        """
        new_size = self.size * 2 
        new_table = [[] for _ in range(new_size)]

        #Rehash all itemns in hash table
        for bucket in self.table:
            for key, value in bucket:
                new_index = self._hash(key) % new_size #rehash all key-value pairs in hash table bucket 
                new_table[new_index].append([key, value])
        
        log.info(f"Replaced old hash table with new hash table and resized the hash table!")
        self.size = new_size 
        self.table = new_table
        return 

    def set(self, key, value):
        """ 
        Check if item already exist in hash table and update the value 
        If the value does not exist update the hash table with the key-value pair
        Call the resize functon if item exceeds load factor threshold
        """
        index = self._hash(key) 
        found = False
        for pair in self.table[index]:
            if pair[0] == key:
                found = True 
                log.info(f"Updated key '{key}' with value '{value}'.")
                pair[1] = value 
                return     
        if not found:
            self.table[index].append([key, value])   
            self.count += 1
            log.info(f"Update hash table with key-value pair '{key}': '{value}'.")
        
        #Check if elements in self.table exceeds the load_factor_threshold and call the _resize function 
        if self.count / self.size > self.load_factor_threshold:
            self._resize() 

    def get(self, key):
        """ 
        Retrieve item from hash table using using key 
        Return None if key not found 
        """
        index = self._hash(key)
        found = False 
        for pair in self.table[index]:
            if pair[0] == key:
                found = True
                log.info(f"Found element '{key}' in the hash table")
                return pair[1]
        
        if not found:
            log.info(f"Element '{key}' not found in the hash table")
            return None 
        
    def delete(self, key):
        """
        Delete element by key from the hash table  
        """
        index = self._hash(key)

        for i, pair in enumerate(self.table[index]):
            if pair[0] == key:
                del self.table[index][i]
                log.info(f"Deleted element '{key}' from hash table.")
                return 
        
        log.info(f"Element '{key}' not found in the hash table.")
        return None 
    
    def __str__(self):
        """ 
        For Debugging: Return string representation of items in hash table 
        """
        table_str = ""
        for i, bucket in enumerate(self.table):
            if bucket:
                table_str += f"Bucket {i}: " + ", ".join([f"{k}: {v}" for k, v in bucket]) + "\n"
        return table_str if table_str else "Hash Table is empty."
    
if __name__ == '__main__':
    hash_bucks = HashTable(size=10)

    #set item in hash table 
    hash_bucks.set("apple", 1)
    hash_bucks.set("banana", 2)
    hash_bucks.set("orange", 3)
    hash_bucks.set("apple", 10) # update existing key

    #Triger resizing
    hash_bucks.set("grape", 4)
    hash_bucks.set("melon", 5)
    hash_bucks.set("berry", 6)
    hash_bucks.set("peach", 7)

    #Retrieve element by key 
    hash_bucks.get("apple")
    hash_bucks.get("banana")
    hash_bucks.get("grape")

    #Delement item by key 
    hash_bucks.delete("banana")
    hash_bucks.get("banana")

    #Check the current state of the Hash Table 
    print(hash_bucks)

INFO:__main__:Update hash table with key-value pair 'apple': '1'.
INFO:__main__:Update hash table with key-value pair 'banana': '2'.
INFO:__main__:Update hash table with key-value pair 'orange': '3'.
INFO:__main__:Updated key 'apple' with value '10'.
INFO:__main__:Update hash table with key-value pair 'grape': '4'.
INFO:__main__:Update hash table with key-value pair 'melon': '5'.
INFO:__main__:Update hash table with key-value pair 'berry': '6'.
INFO:__main__:Update hash table with key-value pair 'peach': '7'.
INFO:__main__:Found element 'apple' in the hash table
INFO:__main__:Found element 'banana' in the hash table
INFO:__main__:Found element 'grape' in the hash table
INFO:__main__:Deleted element 'banana' from hash table.
INFO:__main__:Element 'banana' not found in the hash table


Bucket 0: apple: 10
Bucket 3: peach: 7
Bucket 6: orange: 3
Bucket 7: grape: 4
Bucket 8: berry: 6
Bucket 9: melon: 5

